In [8]:
import pandas as pd

In [9]:
df = pd.read_csv('../data/US_Accidents_March23.csv')
df.head()

,ID,Source,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,End_Lat,End_Lng,Distance(mi),...,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight
0,A-1,Source2,3,2016-02-08 05:46:00,2016-02-08 11:00:00,39.865147,-84.058723,NaN,NaN,0.01,...,False,False,False,False,False,False,Night,Night,Night,Night
1,A-2,Source2,2,2016-02-08 06:07:59,2016-02-08 06:37:59,39.928059,-82.831184,NaN,NaN,0.01,...,False,False,False,False,False,False,Night,Night,Night,Day
2,A-3,Source2,2,2016-02-08 06:49:27,2016-02-08 07:19:27,39.063148,-84.032608,NaN,NaN,0.01,...,False,False,False,False,True,False,Night,Night,Day,Day
3,A-4,Source2,3,2016-02-08 07:23:34,2016-02-08 07:53:34,39.747753,-84.205582,NaN,NaN,0.01,...,False,False,False,False,False,False,Night,Day,Day,Day
4,A-5,Source2,2,2016-02-08 07:39:07,2016-02-08 08:09:07,39.627781,-84.188354,NaN,NaN,0.01,...,False,False,False,False,True,False,Day,Day,Day,Day


In [10]:
# print rows and cols
df.shape

(7728394, 46)

In [11]:
# shrink df size to 50mb for testing
sample_df = df.sample(frac=0.017, random_state=10)

# fix datetime type
sample_df['Start_Time'] = sample_df['Start_Time'].str.replace(r'\.\d+', '', regex=True)
sample_df['End_Time'] = sample_df['End_Time'].str.replace(r'\.\d+', '', regex=True)
sample_df['Start_Time'] = pd.to_datetime(sample_df['Start_Time'])
sample_df['End_Time'] = pd.to_datetime(sample_df['End_Time'])
sample_df['Weather_Timestamp'] = pd.to_datetime(sample_df['Weather_Timestamp'])

# save to csv
sample_df.to_csv('../data/US_Accidents_March23_sample.csv', index=False)
sample_df.head()

,ID,Source,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,End_Lat,End_Lng,Distance(mi),...,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight
227986,A-227993,Source2,2,2016-05-18 16:28:12,2016-05-18 16:58:12,39.815620,-82.822304,NaN,NaN,0.000,...,False,False,False,False,False,False,Day,Day,Day,Day
5542742,A-5584682,Source1,2,2021-10-17 00:25:30,2021-10-17 00:52:30,35.110082,-80.975220,35.107982,-80.978715,0.245,...,False,False,False,False,False,False,Night,Night,Night,Night
309620,A-309630,Source2,2,2016-11-13 16:55:24,2016-11-13 17:40:24,47.234718,-122.487633,NaN,NaN,0.010,...,False,False,False,False,False,False,Night,Day,Day,Day
3570299,A-3580182,Source1,2,2017-03-15 09:21:23,2017-03-15 15:21:23,43.176570,-71.611080,43.177915,-71.624505,0.683,...,False,False,False,False,False,False,Day,Day,Day,Day
6476052,A-6523880,Source1,2,2021-02-12 04:27:00,2021-02-12 08:44:00,41.944932,-83.555199,41.945482,-83.559469,0.223,...,False,False,False,False,False,False,Night,Night,Night,Night


In [12]:
# print sample rows and cols
sample_df.shape

(131383, 46)